In [18]:
import os.path

import pandas as pd
from os import listdir, path
from pathlib import Path

In [19]:
root_folder = Path("./..").resolve()
datasets_folder = root_folder / 'exploratory_datasets'
clean_data_folder = root_folder / 'cleaned_data'
file = datasets_folder / "qogdata2013.csv"

In [20]:
fos = pd.read_csv(file, sep=";")
report_2022 = pd.read_csv(datasets_folder / "Freedom_of_the_press" / "2022.csv")
report_2023 = pd.read_csv(datasets_folder / "Freedom_of_the_press" / "2023.csv", sep=";")

In [21]:
report_2022 = report_2022.dropna(how="all", axis=0)

In [22]:
report_2022 = report_2022.rename(columns={"ISO": "iso_3", "Score N" : "global_score", "Rank N-1": "country_name" })
report_2022["year"] = 2022
report_2022 = report_2022[['country_name', 'year', 'iso_3', 'global_score']]

In [23]:
report_2023 = report_2023.rename(columns={"ISO": "iso_3", "Score" : "global_score", "Country_EN": "country_name" })
report_2023["year"] = 2023
report_2023 = report_2023[['country_name', 'year', 'iso_3', 'global_score']]
report_2023['global_score'] = pd.to_numeric(report_2023['global_score'].str.replace(",", "."))

In [24]:
fos["year"] += 1
fos = fos.loc[:,['cname', 'year', 'ccodealp', 'rsf_pfi1321']]
fos['rsf_pfi1321'] = pd.to_numeric(fos['rsf_pfi1321'].str.replace(",", "."))
fos = fos.rename(columns={"cname": "country_name", "ccodealp": "iso_3", "rsf_pfi1321": "global_score"})

In [25]:
full_data = pd.concat([fos, report_2022, report_2023])

In [26]:
evolution = (
    full_data
    .sort_values("year")
    .groupby("iso_3", as_index=False)["global_score"]
    .rolling(2)
    .apply(lambda x: x.iloc[1] - x.iloc[0])
    .fillna(0)
    .rename(columns={"global_score": "fos_yearly_evolution", "iso_3": "iso_3_copy"})
)

In [27]:
full_data_with_evolution = full_data.sort_values("year").join(evolution)

In [28]:
full_data_with_evolution = full_data_with_evolution[full_data_with_evolution["iso_3"]==full_data_with_evolution["iso_3_copy"]]

In [29]:
full_data_with_evolution = full_data_with_evolution.drop("iso_3_copy", axis=1)

In [30]:
full_data_with_evolution.to_parquet(clean_data_folder / "freedom_of_the_press_2013_2023.parquet")